# Object Designator
Object designator are used to describe objects located in the BulletWorld or the real environment and then resolve them during runtime to concrete objects.

Object designator are different from the Object class in bullet_world.py in the way that they just describe an object and do not create objects or provide methods to manipulate them. Nethertheless, object designator contain a reference to the BulletWorld object.

Object designator take two parameter, of which at least one has to be provided. These parameter are:

   * A list of names 
   * A list of types 
   
Object Designators work similar to Location designators, they get constrains describing a set of objects and when resolved return a specific instance. 

For all following examples we need a BulletWorld, so let's create one.

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.pose import Pose

world = BulletWorld()

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']


In [7]:
world.exit()

## Believe Object
This object designator is used to describe object that are located in the BulletWorld. So objects that are in the believe state, hence the name. In the futre when there is a perception interface there will be a ```RealObject``` description which will be used to describe objects in the real world. 

Since ```BelieveObject``` decribes Objects in the BulletWorld we create a few.

In [2]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", "breakfast_cereal", "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", "cutlery", "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


Now that we have objects we can create an object designator to describe them. For the start we want an object designator only describing the milk. Since all objects have unique names we can create an object designator using a list with only the name of the object. 

In [3]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(names=["milk"])

print(object_description.resolve())

BelieveObject.Object(name='milk', type='milk', bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1690275327
    nsecs: 172023534
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
base_origin_shift=[ 4.15300950e-04 -6.29518181e-05  8.96554102e-02], 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7f21af118610>, 
tf_frame=milk_3, 
urdf_object: ...), _pose=<bound method Object.get_pose of Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.

You can also use the type to describe objects, so now we want to have an object designator that describes every food in the world. 

In [4]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(types=["milk", "breakfast_cereal"])

print(object_description.resolve())

BelieveObject.Object(name='milk', type='milk', bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1690275327
    nsecs: 172023534
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
base_origin_shift=[ 4.15300950e-04 -6.29518181e-05  8.96554102e-02], 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7f21af118610>, 
tf_frame=milk_3, 
urdf_object: ...), _pose=<bound method Object.get_pose of Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.

## Object Part 
Part of object designators can be used to describe describe something as part of another obeject. For example, you could describe a specific drawer as part of the kitchen. This is necessary since the drawer is no single BulletWorld Object but rather a link of the kitchen which is a BulletWorld Object.

For this example we need just need the kitchen, if you didn't spawn it in the previous example you can spawn it with the following cell.

In [ ]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")

In [5]:
from pycram.designators.object_designator import ObjectPart, BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()

object_description = ObjectPart(names=["sink_area_left_upper_drawer_main"], part_of=kitchen_desig)

print(object_description.resolve())

ObjectPart.Object(name='sink_area_left_upper_drawer_main', type=None, bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=kitchen, 
type=environment, 
color=[1, 1, 1, 1], 
id=2, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/kitchen.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1690275323
    nsecs: 246471166
  frame_id: "map"
pose: 
  position: 
    x: 0.0
    y: 0.0
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
base_origin_shift=[0.    0.    0.001], 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7f21af118610>, 
tf_frame=kitchen_2, 
urdf_object: ...), _pose=<bound method Object.get_pose of Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=kitchen, 
type=environment, 
color=[1, 1, 1, 1], 
id=2, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../r

## Object Designators as Generators 
Similar to location designators object designators can be used as generators to iterate through every object that they are describing. We will see this at the example of an object designator describing every type of food. 

For this we need some obejcts, so if you didn't already spawn them you can use the next cell for this.

In [ ]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", "breakfast_cereal", "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", "cutlery", "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))

In [6]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(types=["milk", "breakfast_cereal"])

for obj in object_description:
    print(obj, "\n")

BelieveObject.Object(name='milk', type='milk', bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1690275327
    nsecs: 172023534
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
base_origin_shift=[ 4.15300950e-04 -6.29518181e-05  8.96554102e-02], 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7f21af118610>, 
tf_frame=milk_3, 
urdf_object: ...), _pose=<bound method Object.get_pose of Object(world=<pycram.bullet_world.BulletWorld object at 0x7f21af0f6d60>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.